<a href="https://colab.research.google.com/github/Alberto-97sc/mmshap_medclip/blob/others-clips-version/notebooks/02_rclip_classification_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RClip + SHAP: Clasificación Médica con Balance Multimodal

Este notebook demuestra:
- **Clasificación de imágenes médicas** usando RClip
- **Análisis de explicabilidad** con SHAP
- **Medición del balance multimodal** (TScore/IScore)
- **Visualización de mapas de calor** para parches de imagen y tokens de texto

Dataset: **ROCO** (Radiology Objects in COntext)


## 🚀 Configuración inicial


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Configuración del repositorio
REPO_URL = "https://github.com/Alberto-97sc/mmshap_medclip.git"
LOCAL_DIR = "/content/mmshap_medclip"
BRANCH = "others-clips-version"  # Rama con RClip

%cd /content
import os, shutil, subprocess, sys

if not os.path.isdir(f"{LOCAL_DIR}/.git"):
    # No está clonado aún
    !git clone $REPO_URL $LOCAL_DIR
else:
    # Ya existe: actualiza a la última versión del remoto
    %cd $LOCAL_DIR
    !git fetch origin
    !git checkout $BRANCH
    !git reset --hard origin/$BRANCH

%cd $LOCAL_DIR
!git rev-parse --short HEAD


In [ ]:
# Instalar el paquete en modo editable
%pip install -e /content/mmshap_medclip

# Dependencias adicionales si son necesarias
%pip install matplotlib seaborn pillow


## 📊 Carga de datos y modelo


In [ ]:
# Configuración para RClip + clasificación
CFG_PATH = "/content/mmshap_medclip/configs/roco_classification_rclip.yaml"

from mmshap_medclip.io_utils import load_config
from mmshap_medclip.devices import get_device
from mmshap_medclip.registry import build_dataset, build_model

# Cargar configuración
cfg = load_config(CFG_PATH)
device = get_device()
print(f"🖥️ Dispositivo: {device}")

# Cargar dataset ROCO
print("📁 Cargando dataset ROCO...")
dataset = build_dataset(cfg["dataset"])
print(f"✅ Dataset cargado: {len(dataset)} muestras")

# Cargar modelo RClip
print("🤖 Cargando modelo RClip...")
model = build_model(cfg["model"], device=device)
print("✅ Modelo RClip cargado")


In [ ]:
# Definir clases para clasificación médica
class_names = [
    "Chest X-Ray",
    "Brain MRI", 
    "Abdominal CT Scan",
    "Ultrasound",
    "OPG",  # Orthopantomogram
    "Mammography",
    "Bone X-Ray",
    "Cardiac MRI",
    "Pulmonary CT",
    "Spinal MRI"
]

print(f"🏷️ Clases definidas: {len(class_names)}")
for i, clase in enumerate(class_names):
    print(f"  {i+1}. {clase}")


## 🔍 Ejemplo 1: Clasificación simple (sin SHAP)


In [ ]:
from mmshap_medclip.tasks.classification import run_classification_one
import matplotlib.pyplot as plt

# Seleccionar una muestra del dataset
muestra_idx = 266  # Cambiar por cualquier índice válido
sample = dataset[muestra_idx]
image = sample['image']
caption = sample['text']

print(f"📋 Muestra {muestra_idx}:")
print(f"Caption original: {caption[:100]}...")

# Mostrar la imagen
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.title(f"Muestra {muestra_idx} - ROCO Dataset")
plt.axis('off')
plt.show()


In [ ]:
# Clasificación rápida sin SHAP
print("⚡ Ejecutando clasificación rápida (sin SHAP)...")
res_simple = run_classification_one(
    model, image, class_names, device, 
    explain=False  # Sin explicabilidad para mayor velocidad
)

print(f"\n🎯 Resultados de clasificación:")
print(f"Clase predicha: {res_simple['predicted_class']}")
print(f"Confianza: {res_simple['probabilities'].max():.2%}")

print(f"\n📊 Todas las probabilidades:")
for clase, prob in zip(class_names, res_simple['probabilities']):
    bar = "█" * int(prob * 20)  # Barra visual
    print(f"  {clase:<20}: {prob:.2%} {bar}")


## 🧠 Ejemplo 2: Clasificación con SHAP y Balance Multimodal


In [ ]:
print("🔬 Ejecutando clasificación con SHAP (esto puede tomar varios minutos)...")
res_shap = run_classification_one(
    model, image, class_names, device, 
    explain=True,  # Con explicabilidad SHAP
    plot=True      # Generar mapas de calor
)

print(f"\n🎯 Resultados con SHAP:")
print(f"Clase predicha: {res_shap['predicted_class']}")
print(f"Confianza: {res_shap['probabilities'].max():.2%}")

print(f"\n⚖️ Balance Multimodal:")
print(f"TScore (Text Score): {res_shap['tscore']:.2%}")
print(f"IScore (Image Score): {res_shap['iscore']:.2%}")

# Interpretación del balance
if res_shap['tscore'] > 0.6:
    balance_msg = "🔤 Modelo se enfoca más en el TEXTO"
elif res_shap['iscore'] > 0.6:
    balance_msg = "🖼️ Modelo se enfoca más en la IMAGEN"
else:
    balance_msg = "⚖️ Balance equilibrado entre texto e imagen"
    
print(f"Interpretación: {balance_msg}")


In [ ]:
# Mostrar mapa de calor si está disponible
if 'fig' in res_shap:
    print("🗺️ Mapa de calor con importancia de parches y tokens:")
    display(res_shap['fig'])
else:
    print("⚠️ No se generó mapa de calor")


## 🎉 Conclusiones

Este notebook ha demostrado:

1. **✅ Clasificación médica**: RClip puede clasificar imágenes médicas en múltiples categorías
2. **✅ Explicabilidad con SHAP**: Podemos entender qué partes de la imagen y texto son importantes
3. **✅ Balance multimodal**: Medimos si el modelo se enfoca más en texto o imagen
4. **✅ Visualización**: Mapas de calor muestran la importancia espacial y textual

### Próximos pasos:
- Evaluar en más muestras del dataset
- Comparar con otros modelos CLIP
- Análisis de casos específicos por tipo de imagen médica
- Optimización de hiperparámetros para mejor balance multimodal
